In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
import category_encoders as ce
from sklearn.model_selection import train_test_split

In [23]:
data=pd.DataFrame([
    ['nuc','youn','low','yes'],
    ['ext','old','low','no'],
    ['less','mid','low','no'],
    ['less','youn','mid','yes'],
    ['sin','mid','mid','yes'],
    ['les','youn','low','no'],
    ['nuc','old','high','yes'],
    ['nuc','mid','mid','yes'],
    ['ext','mid','high','yes'],
    ['sin','old','low','no'],
], columns=['fam', 'age', 'inc', 'targ'])

prob=[
    ['sin','mid','high',],
    ['less','old','low',],
    ['ext','youn','mid',],
    ['nuc','youn','high',],
    ['nuc','youn','low',],
]

In [24]:
mod={targ: {col: {} for col in data.columns if col != 'targ'} for targ in ['yes', 'no']}
for targ in ['yes', 'no']:
    df = data[data['targ']==targ]
    for col in df.columns:
        if col == 'targ': continue
        for v in df[col].unique():
            mod[targ][col][v]=len(df[df[col]==v])/len(df)
    print(targ,len(df))
mod

yes 6
no 4


{'yes': {'fam': {'nuc': 0.5,
   'less': 0.16666666666666666,
   'sin': 0.16666666666666666,
   'ext': 0.16666666666666666},
  'age': {'youn': 0.3333333333333333, 'mid': 0.5, 'old': 0.16666666666666666},
  'inc': {'low': 0.16666666666666666, 'mid': 0.5, 'high': 0.3333333333333333}},
 'no': {'fam': {'ext': 0.25, 'less': 0.25, 'les': 0.25, 'sin': 0.25},
  'age': {'old': 0.5, 'mid': 0.25, 'youn': 0.25},
  'inc': {'low': 1.0}}}

In [41]:
def pred(x,mo):
    res={}
    for targ in ['yes', 'no']:
        df = data[data['targ']==targ]
        res[targ]=len(df)/len(data)
        for i in range(len(df.columns)-1):
            try:
                res[targ]*=mo[targ][df.columns[i]][x[i]]
            except KeyError:
                res[targ]*=0
    res=[list(res.keys()),list(res.values())]
    return res[0][np.argmax(res[1])]
for xx in prob:
    print(xx, pred(xx, mod))

['sin', 'mid', 'high'] yes
['less', 'old', 'low'] no
['ext', 'youn', 'mid'] yes
['nuc', 'youn', 'high'] yes
['nuc', 'youn', 'low'] yes
